# Notebook Title TBD

Version 0.00001

## Introduction 

TBD

## <a id = "EnvironmentPreparation">Environment preparation</a>

In [ ]:
########## Environment preparation ############

# Set Student ID number
stdid={{ STDID }}
Id=$(id --user --name)

# OneView variables
OneViewSimulator="STA1-HPEOneView-DCS_5.40"
#OVVMSIMU1: "STA1-HPEOneView-DCS_5.40"

OneViewSimulatorIP="16.31.86.247"
#OVVMSIMU1IP: "16.31.86.247"

OneViewSimulatorName="ov31"
#OVVMSIMU1NAME: "ov31"

OneViewSimulatorFQDN="ov31.etc.fr.comm.hpecorp.net"
#OVVMSIMU1FQDN: "ov31.etc.fr.comm.hpecorp.net"

